In [1]:
from pprint import pprint
from lxml import html
import requests
import datetime,time
import unicodedata
import json
import pandas as pd

In [2]:
yandex_link = 'https://yandex.ru/news/'
mailru_link = 'https://news.mail.ru/'
lenta_link = 'https://lenta.ru/'

In [3]:
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
yandex_request = requests.get(yandex_link,headers=header).text
mailru_request = requests.get(mailru_link,headers=header).text
lenta_request = requests.get(lenta_link,headers=header).text

yandex_html = html.fromstring(yandex_request)
mailru_html = html.fromstring(mailru_request)
lenta_html = html.fromstring(lenta_request)

### YANDEX NEWS

In [4]:
news_link_yandex = yandex_html.xpath('//h2/a[contains(@class, link_theme_black)]/@href')
news_header_yandex = yandex_html.xpath('//h2/a[contains(@class, link_theme_black)]/text()')
news_origin_date_yandex = yandex_html.xpath('//div[@class = "story__date"]/text()')

news_origin_date_yandex1 = news_origin_date_yandex.copy()
for k,i in enumerate(news_link_yandex):
    news_link_yandex[k] = i.replace('/news/',yandex_link)
for k,i in enumerate(news_origin_date_yandex):
    news_origin_date_yandex[k] = unicodedata.normalize('NFKD',i).split(' ')
    news_origin_date_yandex[k][0] = ' '.join(news_origin_date_yandex[k][:-1])
    if str('вчера в') in news_origin_date_yandex[k][0]:
        news_origin_date_yandex[k][0]=news_origin_date_yandex[k][0].replace('вчера в','')
        news_origin_date_yandex[k][-1] = 'вчера в ' + news_origin_date_yandex[k][-1]

for z,k in enumerate([i[-1] for i in news_origin_date_yandex]):
    if len(k)==5:
        news_origin_date_yandex[z][-1] = str(datetime.date.today())
    elif 'вчера' in str.lower(k):
        news_origin_date_yandex[z][-1] = str(datetime.date.today()-datetime.timedelta(days=1))

for i in zip(news_header_yandex,news_link_yandex,[i[0] for i in news_origin_date_yandex],[i[-1] for i in news_origin_date_yandex]):
    #print (i)
    pass

### LENTA NEWS

In [5]:
news_link_lenta = lenta_html.xpath('//div[@class = "item"]//a/@href|//div[@class = "item news b-tabloid__topic_news"]//h3/a/@href')
news_header_lenta = lenta_html.xpath('//div[@class = "item"]//a/text()|//div[@class = "item news b-tabloid__topic_news"]//h3/a/span/text()')

for k,i in enumerate(news_header_lenta):
    news_header_lenta[k]=unicodedata.normalize('NFKD',i)
news_date_lenta = []
for i in news_link_lenta:
    if len(i.split('.htm'))>1:
        news_date_lenta.append('/'.join(''.join(list(i.split('.htm')[0])[-10:]).split('-')[::-1]))
    else:
        news_date_lenta.append('/'.join(i.split('/')[2:5]))
news_origin_lenta = []
for k,i in enumerate(news_link_lenta):
    if str(i)[0] == '/':
        news_origin_lenta.append('Lenta.Ru')
    else:
        news_origin_lenta.append(i.replace('https://','').split('/')[0].title())

for i in zip(news_header_lenta,news_link_lenta,news_origin_lenta,news_date_lenta):
    #print (i)
    pass

### MAIL.RU NEWS

In [6]:
news_link_mailru = mailru_html.xpath('//div[contains(@class,"daynews__item")]//a/@href|//a[@class="list__text"]/@href|//a[@class="link link_flex"]/@href|//a[@class="newsitem__title link-holder"]/@href')
news_header_mailru = mailru_html.xpath('//div[contains(@class,"daynews__item")]//a//span[contains(@class,"photo__title_new")]/text()|//a[@class="list__text"]/text()|//a[@class="link link_flex"]/span/text()|//a[@class="newsitem__title link-holder"]/span/text()')

for k,i in enumerate(news_header_mailru):
    news_header_mailru[k]=unicodedata.normalize('NFKD',i)
for k,i in enumerate(news_link_mailru):
    if str(i)[0] == '/':
        news_link_mailru[k] = mailru_link + str(i)[1:]
    else:
        continue
# если - вдруг - источник не найдется (или не указан, как в lenta.ru), примем, что источником является mail.ru
news_origin_mailru = []
for i in news_link_mailru:
    news_origin_mailru.append(i.replace('https://','').split('/')[0].title())

news_date_mailru = []
for k,i in enumerate(news_link_mailru):
    time.sleep(1)
    news_date_request = html.fromstring(requests.get(i,headers = header).text)
    news_date = news_date_request.xpath('//span[contains(@class,"note__text breadcrumbs__text js-ago")]/@datetime')
    news_origin_mailru[k]=news_date_request.xpath('//span[@class="breadcrumbs__item"]//span[@class="link__text"]/text()')[0]
    if news_date:
        news_date_mailru.append(str(news_date[0]).split('T')[0])
    else:
        news_date = news_date_request.xpath('//span[contains(@class,"breadcrumbs__item js-ago")]/@datetime')
        if news_date:
            news_date_mailru.append(str(news_date[0]).split('T')[0])
        else:
            news_date = news_date_request.xpath('//time[contains(@class,"breadcrumbs__text js-ago")]/@datetime')
            if news_date:
                news_date_mailru.append(str(news_date[0]).split('T')[0])
            else:
                news_date = json.loads(news_date_request.xpath('//script[@type="application/ld+json"]/text()')[0])['datePublished']
                news_date_mailru.append(news_date)

for i in zip(news_header_mailru,news_link_mailru,news_origin_mailru,news_date_mailru):
    #print (i)
    pass

In [7]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

db = client['db_of_news']

yandex_news_db = db.yandex_news
mailru_db = db.mailru
lenta_db = db.lenta


In [8]:
news_to_add_mailru = {}
k=0
for i in news_header_mailru:
    news_to_add_mailru[k]={'title':news_header_mailru[k],'urls':news_link_mailru[k],'source':news_origin_mailru[k],'date':news_date_mailru[k]}
    k+=1
k=0
news_to_add_lenta = {}
for i in news_header_lenta:
    news_to_add_lenta[k]={'title':news_header_lenta[k],'urls':news_link_lenta[k],'source':news_origin_lenta[k],'date':news_date_lenta[k]}
    k+=1
k=0
news_to_add_yandex = {}
for i in news_header_yandex:
    news_to_add_yandex[k]={'title':news_header_yandex[k],'urls':news_link_yandex[k],'source':[i[0] for i in news_origin_date_yandex][k],'date':[i[-1] for i in news_origin_date_yandex][k]}
    k+=1

In [9]:
for i in news_to_add_mailru:
    mailru_db.insert_one(news_to_add_mailru[i])
for i in news_to_add_lenta:
    lenta_db.insert_one(news_to_add_lenta[i])
for i in news_to_add_yandex:
    yandex_news_db.insert_one(news_to_add_yandex[i])

In [10]:
pd.DataFrame(mailru_db.find())

,_id,title,urls,source,date
0,5e728072f797613bd6c1da40,ВОЗ похвалила Россию за меры против коронавируса,https://news.mail.ru/society/41003381/,ТАСС,2020-03-18
1,5e728072f797613bd6c1da41,В Китае рассказали об эффективном лекарстве от...,https://news.mail.ru/politics/41003826/,РИА Новости,2020-03-18
2,5e728072f797613bd6c1da42,В России обсудят введение нового режима занятости,https://news.mail.ru/politics/41003391/,Известия,2020-03-18
3,5e728072f797613bd6c1da43,ФАС ответила на критику Путина из-за роста цен...,https://news.mail.ru/economics/41002260/,ТАСС,2020-03-18
4,5e728072f797613bd6c1da44,Назван срок жизни коронавируса на разных предм...,https://news.mail.ru/society/40998899/,РИА Новости,2020-03-18
5,5e728072f797613bd6c1da45,ФАС ответила на критику Путина из-за роста цен...,https://news.mail.ru/economics/41002260/,ТАСС,2020-03-18
6,5e728072f797613bd6c1da46,Назван срок жизни коронавируса на разных предм...,https://news.mail.ru/society/40998899/,РИА Новости,2020-03-18
7,5e728072f797613bd6c1da47,Доходы банков России по кредитам выросли на 227%,https://news.mail.ru/economics/41002725/,Известия,2020-03-18
8,5e728072f797613bd6c1da48,Группа Little Big прокомментировала отмену «Ев...,https://news.mail.ru/society/41002512/,РИА Новости,2020-03-18
9,5e728072f797613bd6c1da49,Ученые сообщили о первых испытаниях вакцины от...,https://news.mail.ru/society/41003066/,Известия,2020-03-18
